In [13]:
using WordEmbeddings
using Training

LoadError: LoadError: LoadError: LoadError: syntax: extra token "prob" after end of expression
while loading /mnt_volume/phd/prototypes/SenseSplittingWord2Vec/src/Query.jl, in expression starting on line 1
while loading /mnt_volume/phd/prototypes/SenseSplittingWord2Vec/src/Training.jl, in expression starting on line 10
while loading In[13], in expression starting on line 2

In [14]:

data_dir = joinpath("data") #For local run from testing directory

test_filename = "text8_miniscule"
test_file = joinpath(data_dir, test_filename)


model_file = "models/text8_miniscule.model"

"models/text8_miniscule.model"

In [15]:
using Utils

In [57]:
function test_sense_embedding(inputfile)

	embed = WordSenseEmbedding(30, random_inited, huffman_tree, subsampling = 0, iter=2, strength=0.4, force_minibatch_size=100)
	@time train(embed, inputfile)

    save(embed, model_file)
	embed
end


test_sense_embedding (generic function with 1 method)

In [ ]:
Profile.clear()  # in case we have any previous profiling data
@profile test_sense_embedding(test_file)

In [ ]:
using ProfileView

ProfileView.view()

In [16]:
using SoftmaxClassifier

In [58]:
function test(fn)
    embed = restore(model_file)
    words = split(open(readstring,test_file,"r"))
    time_total = 0.0
    for ii in 10:length(words)-1000
        word = words[ii]
        !haskey(embed.codebook,word) && continue 
        context = [words[ii-5:ii-1]; words[ii+1:ii+5]]
        context = filter(w->haskey(embed.codebook,w),context)
        tic()
        fn(embed,word, context)
        time_total+=toq()
    end
    time_total
end


test (generic function with 2 methods)

In [59]:
function WSD2{S<:AbstractString}(embed::WordEmbeddings.WordSenseEmbedding, word::AbstractString, context::AbstractVector{S})
    function prob_of_context{S<:AbstractString}(embed::GenWordEmbedding, context::AbstractVector{S}, input::Vector{Float32})
        total_logprob=0.0 #Work in logprob to avoid underflow, and get more stability
        for target_word in context
            # discard words not presenting in the classification tree
            haskey(embed.codebook, target_word) || continue
            node = embed.classification_tree      

            word_logprob = 0.0
            for code in embed.codebook[target_word]  
                word_logprob+=log(predict(node.data, input)[code])
                node = node.children[code]
            end
            total_logprob+=word_logprob
        end
        exp(total_logprob) #Going back out of the log domain is not required for external logic, but it is nice for clarity
    end
    
    
    sense_embeddings = embed.embedding[word]
    prob, most_likely_sense_id = findmax([prob_of_context(embed, context, input) for input in sense_embeddings])
    return most_likely_sense_id
end


WSD2 (generic function with 1 method)

In [60]:
gc()
@time test(WSD2)

  3.448796 seconds (14.17 M allocations: 758.525 MB, 5.53% gc time)


3.0366613659999997

In [61]:
@fastmath function WSD3{S<:AbstractString}(embed::WordEmbeddings.WordSenseEmbedding, word::AbstractString, context::AbstractVector{S})
    function prob_of_context{S<:AbstractString}(embed::GenWordEmbedding, context::AbstractVector{S}, input::Vector{Float32})
        total_prob=0.0 #Work in logprob to avoid underflow, and get more stability
        @inbounds for target_word in context
            # discard words not presenting in the classification tree
            haskey(embed.codebook, target_word) || continue
            node = embed.classification_tree      

            word_prob = 0.0
            @inbounds for code in embed.codebook[target_word]  
                word_prob*=predict(node.data, input)[code]
                node = node.children[code]
            end
            total_prob*=word_prob
        end
        exp(total_prob) #Going back out of the log domain is not required for external logic, but it is nice for clarity
    end
    
    
    sense_embeddings = embed.embedding[word]
    prob, most_likely_sense_id = findmax([prob_of_context(embed, context, input) for input in sense_embeddings])
    return most_likely_sense_id
end


WSD3 (generic function with 1 method)

In [62]:
gc()
@time test(WSD3)

  3.237457 seconds (14.16 M allocations: 758.139 MB, 6.06% gc time)


2.8264243710000008

In [63]:
using StatsFuns
import SoftmaxClassifier.predict

In [64]:
@inbounds @inline function softmax5!{R<:AbstractFloat}(t::AbstractArray{R})
    u = max(t[1],t[2])
    @inbounds t[1] = exp(t[1] - u)
    @inbounds t[2] = exp(t[2] - u)
    @inbounds u = t[1]+t[2]
    @inbounds t[1]/=u
    @inbounds t[2]/=u
    t
end
@fastmath function predict5{F<:AbstractFloat}(c::LinearClassifier, x::AbstractVector{F})
    t= F[0., 0.]
    @inbounds for ii in 1: size(c.weights,1)
        @inbounds t[1]+=c.weights[ii,1]*x[ii]
        @inbounds t[2]+=c.weights[ii,2]*x[ii]
    end
    t
    return softmax5!(t)
end

@fastmath function WSD5{S<:AbstractString}(embed::WordEmbeddings.WordSenseEmbedding, word::AbstractString, context::AbstractVector{S})

    
    function prob_of_context{S<:AbstractString}(embed::GenWordEmbedding, context::AbstractVector{S}, input::Vector{Float32})
        total_prob=0.0 #Work in logprob to avoid underflow, and get more stability
        @inbounds for target_word in context
            # discard words not presenting in the classification tree
            haskey(embed.codebook, target_word) || continue
            node = embed.classification_tree      

            word_prob = 0.0
            @inbounds for code in embed.codebook[target_word]  
                word_prob*=predict5(node.data, input)[code]
                node = node.children[code]
            end
            total_prob*=word_prob
        end
        total_prob #Going back out of the log domain is not required for external logic, but it is nice for clarity
    end
        
    sense_embeddings = embed.embedding[word]
    prob, most_likely_sense_id = findmax([prob_of_context(embed, context, input) for input in sense_embeddings])
    return most_likely_sense_id
end


WSD5 (generic function with 1 method)

In [65]:
gc()
@time test(WSD5)

  8.032951 seconds (3.72 M allocations: 333.225 MB, 0.92% gc time)


7.645712660000006

In [96]:
@fastmath @inline function softmax2{R<:AbstractFloat}(t1::R,t2::R)
    u = max(t1,t2)
    t1 = exp(t1 - u)
    t2 = exp(t2 - u)
    s = t1+t2
    t1/=s
    t2/=s
    t1,t2
end
@fastmath function predict{F<:AbstractFloat}(c::LinearClassifier{2}, x::AbstractVector{F})
    t1=zero(F)
    t2=zero(F)
    @inbounds for ii in 1: size(c.weights,1)
        @inbounds t1+=c.weights[ii,1]*x[ii]
        @inbounds t2+=c.weights[ii,2]*x[ii]
    end
    #return softmax!([t1,t2])
    return softmax2(t1,t2)
end

@fastmath function WSD6{S<:AbstractString}(embed::WordEmbeddings.WordSenseEmbedding, word::AbstractString, context::AbstractVector{S})

    
    function prob_of_context{S<:AbstractString}(embed::GenWordEmbedding, context::AbstractVector{S}, input::Vector{Float32})
        total_prob=0.0 #Work in logprob to avoid underflow, and get more stability
        @inbounds for target_word in context
            # discard words not presenting in the classification tree
            haskey(embed.codebook, target_word) || continue
            node = embed.classification_tree      

            word_prob = 0.0
            @inbounds for code in embed.codebook[target_word]  
                word_prob*=predict6(node.data, input)[code]
                node = node.children[code]
            end
            total_prob*=word_prob
        end
        total_prob #Going back out of the log domain is not required for external logic, but it is nice for clarity
    end
        
    sense_embeddings = embed.embedding[word]
    prob, most_likely_sense_id = findmax([prob_of_context(embed, context, input) for input in sense_embeddings])
    return most_likely_sense_id
end


WSD6 (generic function with 1 method)

In [99]:
gc()
@time test(WSD6)

  1.726085 seconds (3.62 M allocations: 62.971 MB, 1.19% gc time)


1.440744877999999

In [90]:
@generated function softmax7!{K,R<:AbstractFloat}(ts::NTuple{K,R})
    quote
        u = maximum(ts)
        @nexprs $K j->t_j = exp(ts[j]-u)
        s=0.0
        @nexprs $K j->s+=t_j
        @nexprs $K j->t_j/=s
        @ntuple $K t
    end
end
@generated function predict7{K,F<:AbstractFloat}(c::LinearClassifier{K}, x::AbstractVector{F})
    quote
        @inbounds begin 
            @nexprs $K j->t_j = 0.0f0
            @simd for ii in 1: size(c.weights,1)
                @nexprs $K j->t_j = c.weights[ii,j]*x[j]
            end
            return softmax!(@ntuple $K t)
        end
    end
end
@inbounds function prob_of_context{S<:AbstractString}(embed::GenWordEmbedding, context::AbstractVector{S}, input::Vector{Float32})
    total_prob=0.0 #Work in logprob to avoid underflow, and get more stability
    for target_word in context
        # discard words not presenting in the classification tree
        haskey(embed.codebook, target_word) || continue
        node = embed.classification_tree      
        
        
        for code in embed.codebook[target_word]  
            total_prob*=predict7(node.data, input)[code]
            node = node.children[code]
        end
    end
    total_prob #Going back out of the log domain is not required for external logic, but it is nice for clarity
end

@fastmath function WSD7{S<:AbstractString}(embed::WordEmbeddings.WordSenseEmbedding, word::AbstractString, context::AbstractVector{S})       
    sense_embeddings = embed.embedding[word]
    prob, most_likely_sense_id = findmax([prob_of_context(embed, context, input) for input in sense_embeddings])
    return most_likely_sense_id
end


WSD7 (generic function with 1 method)

In [92]:
gc()
@time test(WSD7)

  2.272414 seconds (10.59 M allocations: 169.394 MB, 1.49% gc time)


1.9885797159999994

In [28]:
@generated function softmax8!{K,R<:AbstractFloat}(ts::AbstractVector{R}, ::Type{Val{K}})
    quote
        u = maximum(ts)
        @nexprs $K j->t_j = exp(ts[j]-u)
        s=0.0
        @nexprs $K j->s+=t_j
        @nexprs $K j->+t_j
        @nexprs $K j->t_j/=s
        @ntuple $K t
    end
end

@fastmath function predict8{F<:AbstractFloat}(c::LinearClassifier, x::AbstractVector{F})
    t1=zero(F)
    t2=zero(F)
    x_len = length(x)
    @simd for ii in 1:x_len
        @inbounds t1+=c.weights[ii,1]*x[ii]
        @inbounds t2+=c.weights[ii,2]*x[ii]
    end
    return softmax8!(t1,t2)
end
@inbounds function prob_of_context8{S<:AbstractString}(embed::GenWordEmbedding, context::AbstractVector{S}, input::Vector{Float32})
    total_prob=0.0 #Work in logprob to avoid underflow, and get more stability
    for target_word in context
        # discard words not presenting in the classification tree
        haskey(embed.codebook, target_word) || continue
        node = embed.classification_tree      
        
        
        for code in embed.codebook[target_word]  
            total_prob*=predict8(node.data, input)[code]
            node = node.children[code]
        end
    end
    total_prob #Going back out of the log domain is not required for external logic, but it is nice for clarity
end

@fastmath function WSD8{S<:AbstractString}(embed::WordEmbeddings.WordSenseEmbedding, word::AbstractString, context::AbstractVector{S})       
    sense_embeddings = embed.embedding[word]
    prob, most_likely_sense_id = findmax([prob_of_context8(embed, context, input) for input in sense_embeddings])
    return most_likely_sense_id
end


WSD8 (generic function with 1 method)

In [29]:
gc()
@time test(WSD8)

LoadError: LoadError: MethodError: no method matching softmax8!(::Float32, ::Float32)
while loading In[29], in expression starting on line 155

In [45]:
methods(StatsFuns.softmax!)

# 2 methods for generic function "softmax!":
softmax!{R<:AbstractFloat,T<:Real}(r::AbstractArray{R,N<:Any}, x::AbstractArray{T,N<:Any}) at /home/ubuntu/.julia/v0.5/StatsFuns/src/basicfuns.jl:147
softmax!{T<:AbstractFloat}(x::AbstractArray{T,N<:Any}) at /home/ubuntu/.julia/v0.5/StatsFuns/src/basicfuns.jl:161

In [ ]:
using ProfileView
Profile.clear()
@profile test(WSD7)
ProfileView.view()


In [ ]:
using ProfileView
Profile.clear()
@profile test(WSD6)
ProfileView.view()


In [ ]:
using ProfileView
Profile.clear()
@profile test(WSD5)
ProfileView.view()


In [ ]:
using ProfileView
Profile.clear()
@profile test(WSD3)
ProfileView.view()


In [ ]:
?max

In [1]:
using Base.Cartesian
using StatsFuns

In [105]:
inv(2.f0)

0.5f0

In [2]:
@fastmath function softmax2!{R<:AbstractFloat}(t1::R,t2::R)
    u = max(t1,t2)
    t1 = exp(t1 - u)
    t2 = exp(t2 - u)
    s = t1+t2
    t1/=s
    t2/=s
    t1,t2
end

softmax2! (generic function with 1 method)

In [36]:
@generated function softmaxK!{K,R<:AbstractFloat}(ts::NTuple{K,R})
    quote
        u = maximum(ts)
        @nexprs $K j->t_j = exp(ts[j]-u)
        s=0.0
        @nexprs $K j->s+=t_j
        @nexprs $K j->t_j/=s
        @ntuple $K t
    end
end

softmaxK! (generic function with 1 method)

In [44]:
@generated function softmaxKf!{K,R<:AbstractFloat}(ts::NTuple{K,R})
    quote
        u = maximum(ts)
        @nexprs( $K j-> t_j = @fastmath(exp(ts[j]-u))
        s=0.0
        @nexprs( $K j->( s+=t_j)
        @nexprs( $K j->( t_j/=s)
        @ntuple $K t
    end
end

LoadError: LoadError: syntax: missing comma or ) in argument list
while loading In[44], in expression starting on line 1

In [4]:
@generated function softmaxKAB!{K,R<:AbstractFloat}(ts::AbstractVector{R}, ::Type{Val{K}})
    quote
        u = maximum(ts)
        @nexprs $K j->t_j = exp(ts[j]-u)
        s=0.0f0
        @nexprs $K j->s+=t_j
        @nexprs $K j->t_j/=s
        @ntuple $K t
    end
end

softmaxKAB! (generic function with 1 method)

In [213]:
for ii in 1:1000
    v=rand(2)
    s1 =  softmax(v)
    s2 = softmax2a!(v[1],v[2])
    @assert(s1[1]≈s2[1], "$(s1[1]) !=  $(s2[1])")
    @assert(s1[2]≈s2[2], "$(s1[2]) !=  $(s2[2])")
end


In [256]:
@fastmath function softmax2a!{R<:AbstractFloat}(t1::R,t2::R)
    
    if t1>t2
        z=t2-t1
        m  = exp(z)
        r1 = inv(one(R)+m)
        r2 = m*r1
        (r1,r2)
    else #t2>t1
        z=t1-t2
        m  = exp(z)
        r2 = inv(one(R)+m)
        r1 = m*r2
    end
    (r1,r2)

end
@code_native softmax2a!(0.25,0.25)

	.text
Filename: In[256]
Source line: 0
	pushq	%rbp
	movq	%rsp, %rbp
	pushq	%rbx
	pushq	%rax
	movq	%rdi, %rbx
	movabsq	$exp, %rax
Source line: 3
	vucomisd	%xmm1, %xmm0
	jbe	L59
Source line: 4
	vsubsd	%xmm0, %xmm1, %xmm0
Source line: 5
	callq	*%rax
	movabsq	$140639058038784, %rax  # imm = 0x7FE915189000
	vmovsd	(%rax), %xmm1           # xmm1 = mem[0],zero
Source line: 6
	vaddsd	%xmm1, %xmm0, %xmm2
Source line: 231
	vdivsd	%xmm2, %xmm1, %xmm1
Source line: 7
	vmulsd	%xmm0, %xmm1, %xmm2
Source line: 8
	jmp	L91
Source line: 10
L59:
	vsubsd	%xmm1, %xmm0, %xmm0
Source line: 11
	callq	*%rax
	movabsq	$140639058038784, %rax  # imm = 0x7FE915189000
	vmovsd	(%rax), %xmm1           # xmm1 = mem[0],zero
Source line: 12
	vaddsd	%xmm1, %xmm0, %xmm2
Source line: 231
	vdivsd	%xmm2, %xmm1, %xmm2
Source line: 13
	vmulsd	%xmm0, %xmm2, %xmm1
Source line: 15
L91:
	vmovsd	%xmm2, 8(%rbx)
	vmovsd	%xmm1, (%rbx)
	movq	%rbx, %rax
	addq	$8, %rsp
	popq	%rbx
	popq	%rbp
	retq
	nop


In [251]:
@fastmath @inline function softmax2c{R<:AbstractFloat}(t1::R,t2::R)              
    #This is the softmax function, but particularly optimised.                  
    z = exp(t2-t1)                                                              
    r1 = inv(one(R)+z)                                                          
    r2 = z*r1                                                                  
    (r1,r2)                                                                     
end       
@code_native softmax2c(0.25,0.25)

	.text
Filename: In[251]
Source line: 0
	pushq	%rbp
	movq	%rsp, %rbp
	pushq	%rbx
	pushq	%rax
	movq	%rdi, %rbx
Source line: 3
	vsubsd	%xmm0, %xmm1, %xmm0
Source line: 272
	movabsq	$exp, %rax
	callq	*%rax
	movabsq	$140639058132992, %rax  # imm = 0x7FE9151A0000
Source line: 4
	vmovsd	(%rax), %xmm1           # xmm1 = mem[0],zero
	vaddsd	%xmm1, %xmm0, %xmm2
Source line: 231
	vdivsd	%xmm2, %xmm1, %xmm1
Source line: 5
	vmulsd	%xmm0, %xmm1, %xmm0
Source line: 6
	vmovsd	%xmm1, (%rbx)
	vmovsd	%xmm0, 8(%rbx)
	movq	%rbx, %rax
	addq	$8, %rsp
	popq	%rbx
	popq	%rbp
	retq
	nopw	%cs:(%rax,%rax)


In [254]:
@code_native softmax!([0.25,0.25],[0.25,0.25] )

	.text
Filename: basicfuns.jl
Source line: 0
	pushq	%rbp
	movq	%rsp, %rbp
	pushq	%r15
	pushq	%r14
	pushq	%r13
	pushq	%r12
	pushq	%rbx
	subq	$72, %rsp
	movq	%rsi, %rbx
	movq	%rdi, %r14
	movabsq	$140639742601600, %r13  # imm = 0x7FE93DE62980
	movq	$0, -56(%rbp)
	movq	$0, -48(%rbp)
	movq	$4, -72(%rbp)
	movabsq	$jl_tls_states, %r15
	movq	(%r15), %rax
	movq	%rax, -64(%rbp)
	leaq	-72(%rbp), %rax
	movq	%rax, (%r15)
Source line: 147
	movq	8(%rbx), %r12
Source line: 229
	cmpq	%r12, 8(%r14)
	jne	L498
Source line: 159
	movabsq	$_mapreduce, %rax
	movq	%rbx, %rdi
	callq	*%rax
	vmovapd	%xmm0, %xmm1
	vmovsd	%xmm1, -88(%rbp)
	xorl	%eax, %eax
Source line: 70
	cmpq	$0, %r12
	cmovgq	%r12, %rax
	cmpq	$0, %rax
	je	L470
	movq	%rax, -104(%rbp)
	movq	%r12, %rax
	movq	%rax, -96(%rbp)
	xorl	%r12d, %r12d
	vxorpd	%xmm2, %xmm2, %xmm2
	cmpq	$0, %rax
	movl	$0, %r15d
	cmovgq	%rax, %r15
	nopl	(%rax,%rax)
Source line: 152
L176:
	vmovsd	%xmm2, -80(%rbp)
	movq	(%rbx), %rax
	vmovsd	(%rax,%r12,8), %xmm0    # xmm0 = mem[0],

In [216]:
@code_warntype softmax2!(0.25,0.25)

Variables:
  #self#::#softmax2!
  t1::Float64
  t2::Float64
  u::Float64
  s::Float64

Body:
  begin  # In[2], line 2:
      u::Float64 = (Base.select_value)((Base.lt_float)(t1::Float64,t2::Float64)::Bool,t2::Float64,t1::Float64)::Float64 # In[2], line 3:
      GenSym(0) = (Base.FastMath.box)(Float64,((top(getfield))(Base.FastMath.Base,:sub_float_fast)::I)(t1::Float64,u::Float64))::Float64 # fastmath.jl, line 272:
      t1::Float64 = (top(ccall))((top(tuple))("exp",Base.FastMath.libm)::Tuple{ASCIIString,ASCIIString},Base.FastMath.Float64,(top(svec))(Base.FastMath.Float64)::SimpleVector,GenSym(0),0)::Float64 # In[2], line 4:
      GenSym(2) = (Base.FastMath.box)(Float64,((top(getfield))(Base.FastMath.Base,:sub_float_fast)::I)(t2::Float64,u::Float64))::Float64 # fastmath.jl, line 272:
      t2::Float64 = (top(ccall))((top(tuple))("exp",Base.FastMath.libm)::Tuple{ASCIIString,ASCIIString},Base.FastMath.Float64,(top(svec))(Base.FastMath.Float64)::SimpleVector,GenSym(2),0)::Float64 # In[2], 

In [219]:
function test()
    for ii in 1:1000
        StatsFuns.softmax!([0.25,0.5])
    end
end
gc()
@time test()

  0.008026 seconds (2.89 k allocations: 180.209 KB)


In [240]:
function testa()
    for ii in 1:100000
        softmax2b!(0.25/ii,1./ii)
    end
end
gc()
@time testa()

  0.010031 seconds (1.98 k allocations: 88.513 KB)


In [242]:
function testa()
    for ii in 1:100000
        softmax2a!(0.25/ii,1./ii)
    end
end
gc()
@time testa()

  0.009845 seconds (1.98 k allocations: 88.513 KB)


In [231]:
function testb()
    for ii in 1:100000
        softmax2!(0.25/ii,1./ii)
    end
end
gc()
@time testb()

  0.011857 seconds (2.02 k allocations: 90.215 KB)


In [38]:
function test()
    for ii in 1:1000
        softmaxK!((0.25,0.5))
    end
end
gc()
@time test()

  0.006289 seconds (1.75 k allocations: 80.685 KB)


In [12]:
function test()
    for ii in 1:1000
        softmaxKAB!([0.25,0.5],Val{2})
    end
end
gc()
@time test()

  0.008434 seconds (4.99 k allocations: 217.851 KB)


In [ ]:
Float32(3)

In [ ]:
a[:]=(4,5,6)

In [103]:
median([1,3,4])

3.0